<a href="https://colab.research.google.com/github/kwonbuyeon/2021-1_basicCarAI/blob/master/project1/project1_%ED%9E%88%EC%B9%98%ED%95%98%EC%9D%B4%ED%82%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**자율주행 택시를 위한 오프라인 택시 잡기 기능 히치하이킹**

(Mediapipe를 활용한 차량 지능 관련 자유 프로젝트)
권부연_20203030

이후 자율주행이 보편화된 시대가 되었을때 스마트폰을 통한 자율주행 콜택시, 자율주행 택시를 위한 정류장 등이 생길 수 있다. 

그리고 스마트폰을 사용하지 못할 때 지금의 택시를 부르듯 특정 행동으로 택시를 잡을 수 있으면 좋을 것이라고 생각했다.그래서 히치하이킹에서 영감을 받아 히치하이킹 동작을 인식할 수 있는 프로그램을 개발하였다.


다만, 이것이 가능하기위해선 자율주행차가 지금의 택시가 그러듯 일정 경로를 반복적으로 순회하거나 돌아다녀야 하겠지만, 이는 히치하이킹에서 뿐만 아니라 더 빠르게 콜택시나 정류장 호출에 반응할 수 있는 장점이 있다고 생각한다. 


사용 오픈소스
> holistic
https://google.github.io/mediapipe/solutions/holistic


* test용 파일 다운 링크


https://github.com/kwonbuyeon/2021-1_basicCarAI/tree/master/project1

-------------

mediapipe 연결 



In [176]:
pip install mediapipe

# 웹캠으로 바로 캡쳐한 사진 사용하기


선택사항) 혹시 바로 사진을 캡쳐하여 사용하고 싶다면 아래 shell를 실행하고 main에서 사진을 처리하는 코드를 활용한 후 파일이름 대신해 take_photo()를 사용하여야한다.

(웹캠 capture 기능) filename대신 take_photo()를 넣으면 바로 Capture한 사진을 사용할 수 있다.


In [168]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

#손 모양 인식 구현부

gijun(손목)과의 거리 차이(linesize값)를 기준으로 손가락을 펼친 여부 파악(isfold) 
point1(손가락 세번째마디), point2(손끝)

In [169]:
def linesize(gijun,point):
  return (point.x-gijun.x)**2+(point.y-gijun.y)**2

def isfold(gijun,point1,point2):
  result = True
  if linesize(gijun,point1) > linesize(gijun,point2):
    result = False
  return result

찾은 손의 따봉 여부 판단

참고)
hands
https://google.github.io/mediapipe/solutions/hands

In [170]:
def ishandHich(hand_landmarks):
  if hand_landmarks != None:#손이 인식되었는가
    if isfold(hand_landmarks.landmark[17],hand_landmarks.landmark[2],hand_landmarks.landmark[4]):#엄지를 추켜세웠는가
          isUnfolding= True
          for i in range(6,19,4):#엄지외의 손가락을 접었는가
            if isfold(hand_landmarks.landmark[0],hand_landmarks.landmark[i],hand_landmarks.landmark[i+2]):
              isUnfolding = False
              break
          if isUnfolding:
            return True
  return False

# 손모양과 손위치로 히치하이킹 여부 추정 구현부

주어진 이미지가 히치하이킹하고 있다 판단되면 히치하이킹하는 손을 글자로 표시해줌

손을 엄덩이 보다 머리에 가깝게 들고있는지 확인한다. 손을 기준으로 머리가 엉덩이보다 가까이 있다면 true를 리턴한다.

In [171]:
def ishandUp(head, hip, hand):
  return isfold(hand,head,hip)

히치하이킹을 할 정도의 손위치에서 따봉을 하고 있다면 좌측 상단에 hich라는 글자를 표시한 이미지를 리턴한다.

In [172]:
import cv2
import mediapipe as mp

def hich(image):
  mp_drawing = mp.solutions.drawing_utils
  mp_holistic = mp.solutions.holistic

# For static images:
  with mp_holistic.Holistic(
    static_image_mode=True,
    model_complexity=2) as holistic:

    image_height, image_width, _ = image.shape
    # Convert the BGR image to RGB before processing.
    results = holistic.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))


    # Draw pose, left and right hands, and face landmarks on the image.
    annotated_image = image.copy()
    mp_drawing.draw_landmarks(
        annotated_image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(
        annotated_image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(
        annotated_image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

      #draw hich
    if results.pose_landmarks:
      pose_landmark=results.pose_landmarks
      #right hand
      word = ""
      if ishandUp(pose_landmark.landmark[0],pose_landmark.landmark[24],pose_landmark.landmark[16]):
        if ishandHich(results.right_hand_landmarks):
          word += "right hitch "
          #print("hitch raght hand!")
      #left hand
      if ishandUp(pose_landmark.landmark[0],pose_landmark.landmark[23],pose_landmark.landmark[15]):
        if ishandHich(results.left_hand_landmarks):
          word += "left hitch"
          #print("hitch left hand!")
      cv2.putText(annotated_image, word, (20, 90), 0, 2, (200,0,0),3)
      

      #make file
      return annotated_image

# main 실행부분

선택사항)비디오를 입력한 경우 hich와 자세를 표시한 비디오를 리턴한다.

In [173]:
def video2segemented_video(video_path):
    #load video
    video = cv2.VideoCapture(video_path)
  
    #setting video options (fps, width, height는 기존 영상과 같게 설정하였습니다.)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    fps = video.get(cv2.CAP_PROP_FPS)
    width = video.get(cv2.CAP_PROP_FRAME_WIDTH)
    height = video.get(cv2.CAP_PROP_FRAME_HEIGHT)
    segVideo = cv2.VideoWriter("/content/checked Video.avi",fourcc,fps,(int(width), int(height)))

    print("start")
    #프레임 이미지 수정 및 저장
    while True :
        ret, frame = video.read()#프레임 이미지 가져오기 (ret : frame이 정산적으로 가져와졌는지, frame : 이미지)
        if not(ret):
            break
        image = hich(frame)#이미지에 차선 그리기
        #cv2_imshow(image)
        segVideo.write(image)#저장할 동영상으로 이미지넣기

    video.release()
    segVideo.release()
    print("Successfully Done! check the file 'checked Video.avi' in your folder")

main


*   이미지 입력 시


> cv2.imwrite('출력 파일 경로', hich(cv2.imread("입력 파일경로")))

웹캠 캡쳐이미지 사용 시 "입력 파일경로"위치에 take_photo() 사용




*   동영상 입력 시


> video_path = "입력 파일경로" 
video2segemented_video(video_path)





In [ ]:
#cv2.imwrite( '/content/checked picture.png',hich(cv2.imread("/content/project1_testpicture.jpg")))
video_path = "/content/project1_testvideo.mp4" # input your video path
video2segemented_video(video_path)

start
